System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 8 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 6 on 6 virtual cores
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 6


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw4`


Status `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw4/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [944b1d66] CodecZlib v0.7.1
  [0b1a1467] KrylovKit v0.6.0
⌃ [bdcacae8] LoopVectorization v0.12.158
  [b51810bb] MatrixDepot v1.0.10
  [295af30f] Revise v3.5.2
  [b8865327] UnicodePlots v3.5.3
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra
  [2f01184e] SparseArrays
  [10745b16] Statistics
Info Packages marked with ⌃ have new versions available and may be upgradable.


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{align*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{align*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

### Q1 Answer:
If $r_{i} = 0$, then $p_{ij} = \frac{1}{n}$. If $r_i \neq 0$, then we have:
$$
\begin{aligned}
p_{ij} = \begin{cases}
\frac{1-p}{n} + \frac{p}{r_i} & \text{if $a_{ij} = 1$} \\
\frac{1-p}{n} & \text{if $a_{ij} = 0$}
\end{cases}
\end{aligned}
$$

So, we can write matrix $\mathbf{P}$ as:
$$
\mathbf{P} = \frac{1}{n}\mathbf{b}\mathbf{1}^T + p\mathbf{R}\mathbf{A}
$$
Where $\mathbf{b}$ is a vector with $b_i = 1-p$ if $r_i > 0$, and $b_i = 1$ if $r_i=0$. $\mathbf{R}$ is a diagonal matrix with $R_{ii} = \frac{1}{r_i}$ if $r_i \neq 0$, and $R_{ii} = 0$ if $r_i=0$.

We can see that the first part of the $\mathbf{P}$ matrix is a rank 1 matrix, and the second part is a sparse matrix.

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [3]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/yuyuanlin/.julia/packages/MatrixDepot/lGEHe/src/MatrixDepot.jl:117
┌ Info: reading database
└ @ MatrixDepot /Users/yuyuanlin/.julia/packages/MatrixDepot/lGEHe/src/download.jl:24


┌ Info: adding metadata...
└ @ MatrixDepot /Users/yuyuanlin/.julia/packages/MatrixDepot/lGEHe/src/download.jl:68
┌ Info: adding svd data...
└ @ MatrixDepot /Users/yuyuanlin/.julia/packages/MatrixDepot/lGEHe/src/download.jl:70


┌ Info: writing database
└ @ MatrixDepot /Users/yuyuanlin/.julia/packages/MatrixDepot/lGEHe/src/download.jl:75
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/yuyuanlin/.julia/packages/MatrixDepot/lGEHe/src/MatrixDepot.jl:119


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [4]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:  

* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

* number of web pages

* number of edges (web links)

* number of dangling nodes (pages with no out links)

* histogram of in-degrees  

* list the top 20 pages with the largest in-degrees?  

* histogram of out-degrees

* which the top 20 pages with the largest out-degrees?

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

### Q3 Answer:
#### Part 1

In [5]:
@show sizeof(A)
@show Base.summarysize(A)

sizeof(A) = 40
Base.summarysize(A) = 53276943


53276943

A takes $53276943$ bytes (around 53 MB). A would takes $916428\times 916428\times 8 = 6718722233472$ bytes, which is about 6.7 TB, if it was converted to a `Matrix{Float64}`.

#### Part 2
The number of web pages is $916428$ (the number of rows and columns of $\mathbf{A}$).

#### Part 3
The number of edges is $5105039$ (the number of non-zero entries of $\mathbf{A}$).

#### Part 4

In [6]:
r = sum(A, dims = 2)
count(iszero, r)

176974

The number of dangling nodes is $176974$ (the number of rows with all zeros).

#### Part 5

In [7]:
using UnicodePlots
histogram(sum(A, dims = 1))

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤███████████████████████████████  916 114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

#### Part 6

In [8]:
println("Index of pages with top 20 in-degree", 
        reverse(sortperm(vec(sum(A, dims = 1))))[1:20])

Index of pages with top 20 in-degree

[537040, 597622, 504141, 751385, 32164, 885606, 163076, 819224, 605857, 828964, 551830, 41910, 558792, 459075, 407611, 213433, 765335, 384667, 173977, 687326]


#### Part 7

In [9]:
histogram(r)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤███████████████████████████████  891 798  
   [ 20.0,  40.0) ┤▊ 22 628                                  
   [ 40.0,  60.0) ┤▏ 1 329                                   
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

#### Part 8

In [10]:
println("Index of pages with top 20 out-degree", 
        reverse(sortperm(vec(sum(A, dims = 2))))[1:20])

Index of pages with top 20 out-degree[506743, 305230, 203749, 768092, 808644, 412411, 600480, 376429, 156951, 885729, 667585, 685696, 282141, 598189, 579315, 411594, 321092, 838279, 302734, 915274]


#### Part 9

In [11]:
spy(A)

           ┌──────────────────────────────┐    
         1 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ > 0
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ < 0
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
   916 428 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
           └──────────────────────────────┘    
           ⠀1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀916 428⠀    
           ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀5 105 039 ≠ 0⠀⠀⠀⠀⠀⠀⠀⠀⠀    

In [12]:
spy(A[1:10000, 1:10000])

          ┌──────────────────────────────┐    
        1 │⠀⠀⠀⢀⠀⠁⠀⢀⡀⡬⢅⠂⠀⠀⠂⢀⢐⡂⠀⠡⠀⢈⠀⠈⠀⠀⠂⠂⢈⡀│ > 0
          │⠀⢄⠀⠠⠱⡈⢨⢀⠘⠀⠄⠸⠄⠀⠀⠨⠀⠀⢊⠀⡡⠠⠀⢐⢁⠀⠁⠂⢠⠀│ < 0
          │⠀⠁⡆⠀⠊⠄⠂⡀⠀⠈⠀⠀⡀⠠⠐⠄⢉⡀⠢⠠⠀⢀⠀⠒⢠⠀⠖⠀⢁⢀│    
          │⡀⠈⠆⠐⠀⠤⠁⠀⣀⠀⡂⠌⡀⠄⠀⠀⠊⠀⡠⢀⢀⢹⠠⠈⢀⢀⠐⠠⠈⡀│    
          │⠀⠁⠂⢠⠐⠂⠀⠬⠠⡌⢄⠄⠀⢀⠀⡀⠰⠂⡀⠈⠀⢀⡀⠀⢀⠌⢀⠨⠐⠁│    
          │⠠⠁⣀⡁⠀⠁⡌⢌⠀⠀⡌⠁⠄⢴⡀⠘⣉⠀⠋⠸⠐⡀⠢⠀⠪⠀⢌⠀⠠⠋│    
          │⢠⠀⠁⠁⠈⠀⠀⠄⢀⠀⢀⡀⠀⢐⡀⠐⠄⠄⠀⠀⡀⠘⢉⡠⢥⠰⡀⡀⠀⠁│    
          │⠘⠀⠐⡀⠀⠀⠀⠐⠠⠀⠂⠔⠀⠄⠀⠀⠀⠁⠄⢀⠀⡜⠐⠨⠹⠀⠰⠄⢑⠀│    
          │⠐⢰⠂⠏⠈⠐⠀⠄⠐⠂⠃⡐⠁⠭⢄⠀⠐⠁⠒⠀⠀⠈⠀⢀⠩⠀⠔⠄⠈⠀│    
          │⠔⡂⠊⠀⠀⡂⠄⢈⠁⠈⠉⡀⠀⠀⠤⠃⠄⠨⠂⠀⢀⢀⡀⠀⢨⠈⠀⢀⢀⠄│    
          │⢈⠀⠈⠂⠀⠆⠁⣐⠉⠀⠅⠠⠔⠙⠠⠀⠠⡀⠂⠠⠐⢘⠢⠀⢗⠀⠜⡈⠀⠀│    
          │⡀⠀⠁⣀⠢⠀⠀⢂⠂⠰⡐⠊⠀⠸⡰⡠⠁⠀⠀⠈⠀⠈⠀⠐⢂⠃⠌⠀⠈⢀│    
          │⡀⠀⠀⢀⠀⠀⠀⡀⡔⠈⡀⠄⠀⠁⠐⠄⠀⠂⠀⠃⠉⠰⠄⠁⠀⠁⠄⠠⢨⠀│    
          │⠲⢐⠡⠀⢘⠇⡐⢀⡀⠈⢀⠁⠠⠀⠀⢄⡀⢥⢠⠄⡀⠁⠀⡀⠀⡀⠠⠂⠅⠐│    
   10 000 │⠚⠀⠀⠲⢃⢒⠠⠁⠐⠘⠄⠂⠀⠘⠂⣚⠀⢐⠀⠀⠐⣸⠀⠀⠂⠂⠉⠁⢱⠁│    
          └──────────────────────────────┘    
          ⠀1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀10 000⠀    
          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀600 ≠ 0⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    

The matrix $\mathbf{A}$ is very sparse, as shown in the figure above. The sub-matrix $\mathbf{A}[1:10000, 1:10000]$ is also very sparse with only 600 non-zero entries, as shown in the figure below.

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

### Q4 Answer:
The `LU` type generated by `LinearAlgebra.lu` function is composed of a matrix of n-dimension and a vector of n-dimension. And there is a baseline memory allocation of around $104$ bytes (see below). So, the memory usage is $916428\times 916428\times 8 + 916428\times 8 + 104 \approx 6.72\times 10^{12}$ bytes, which is around $6.72$ TB.

In [13]:
using LinearAlgebra

test1 = rand(1, 1)
test2 = rand(2, 2)
test3 = rand(3, 3)
@show Base.summarysize(lu(test1)) # 120 = 104 + 8 * 1 + 8 * 1
@show Base.summarysize(lu(test2)) # 152 = 104 + 8 * 4 + 8 * 2
@show Base.summarysize(lu(test3)) # 200 = 104 + 8 * 9 + 8 * 3

dump(lu(test3)) # The LU type is composed of 3 fields: a n*n matrix, a n-vector, and an integer.

Base.summarysize(lu(test1)) = 120
Base.summarysize(lu(test2)) = 152
Base.summarysize(lu(test3)) = 200
LU

{Float64, Matrix{Float64}, Vector{Int64}}
  factors: Array{Float64}((3, 3)) [0.9400581376202671

 0.5843862200404148 0.13289217610034854; 0.36990768951083347 0.5887326597780719 0.6805070258774841; 0.9741268863724971 -0.19551792766042053 0.37464346640196783]
  ipiv: Array{Int64}((3,)) [1, 2, 3]
  info: Int64 0


For the running time, the LU decomposition takes $\frac{2}{3} \times n^3 \approx 5.13\times 10^{17}$ (only consider the leading order here given that $n$ is very large here). 

In [14]:
using Statistics
time = zeros(100)
for i in 1:100
    time[i] = 5.13e17 / LinearAlgebra.peakflops()
end

median(time)

2.1972531285e6

It would take my computer $2.3 \times 10 ^{6}$ seconds to finish the computation, which is around 27 days.

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [15]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T                       # teleportation probability
    n         :: Int                     # number of nodes
    # working arrays
    b         :: Vector{T}
    r         :: Vector{T}
    storage_n :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # pre allocation of working arrays
    outdegree = vec(sum(A, dims = 2))
    b         = zeros(T, n)
    r         = zeros(T, n)
    storage_n = zeros(T, n)
    for i in 1:n
        if outdegree[i] > 0
            r[i] = 1 / outdegree[i]
            b[i] = 1 - telep
        else
            r[i] = 0
            b[i] = 1
        end
    end
    PageRankImPt(A, telep, n, b, r, storage_n)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.telep * M.r[j] * M.A[j, i] - b[j] / M.n

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    
    M.storage_n .= M.r .* v
    mul!(out, transpose(M.A), M.storage_n)
    out .= out .* M.telep
    
    vtb = dot(M.b, v)
    out .= v .- (vtb / M.n) .- out
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent

    mul!(M.storage_n, M.A, v)
    M.storage_n .= M.r .* M.storage_n
    M.storage_n .*= M.telep
    
    out .= 1 / M.n * sum(v) .* M.b
    out .= v .- out .- M.storage_n
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [16]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2023spring/master/hw/hw4/pgrksol.csv.gz", join([pwd(), "/pgrksol.csv.gz"]))

println("pgrksol.csv.gz exists!")

pgrksol.csv.gz exists!


In [17]:
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 ⋮
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [18]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

#@assert transpose(M) * ones(n) ≈ zeros(n)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [19]:
#@assert M * xsol ≈ zeros(n)
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate no memory.

In [20]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 397 samples with 1 evaluation.
 Range (min … max):  11.857 ms …  17.257 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.238 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.426 ms ± 670.930 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▇██▆▆▄▆▅▅▅▃▃                                                
  ▆██████████████▆▄▁▄▄▁▁▁▄▁▁▄▄▄▄▁▄▄▄▄▁▁▄▄▆▁▄▄▄▄▁▄▄▁▁▁▁▁▁▁▁▁▁▁▇ ▇
  11.9 ms       Histogram: log(frequency) by time      15.6 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [21]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 389 samples with 1 evaluation.
 Range (min … max):  12.437 ms …  14.908 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.608 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.670 ms ± 267.355 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▃▆▅█▆█▄                                                    
  ▄▃█████████▇▆▅▃▃▃▃▂▁▂▁▃▁▂▄▁▃▁▂▃▂▁▂▁▁▂▁▁▂▁▁▂▁▂▁▁▁▂▁▁▁▁▁▂▁▁▁▃▂ ▃
  12.4 ms         Histogram: frequency by time         13.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [22]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are about 10 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [23]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.3783428219712915e-5, 2.071015539115345e-6, 3.6630659849988166e-6, 7.527510785100929e-7, 8.633285996589004e-7, 1.7694182526211368e-6, 2.431230382745175e-7, 6.368417180325497e-7, 4.744973703426226e-7, 2.6895486109120415e-7  …  3.1679469737736636e-6, 9.72688040959891e-7, 6.588614478101955e-7, 7.737011774201674e-7, 2.431230382745175e-7, 1.6219204213180983e-6, 3.9121300603674997e-7, 2.431230382745175e-7, 7.296033830835416e-6, 6.330939996249744e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821525125666183e-13,).
), time = 3.71809275, bytes = 1010950951, gctime = 0.035640334, gcstats = Base.GC_Diff(1010950951, 137, 0, 203683, 3, 138, 35640334, 4, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [24]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [25]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 3-4 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [33]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[7.002779609160429e-13 + 0.0im], Vector{ComplexF64}[[0.005635826953807464 + 0.0im, 0.0003454914380783898 + 0.0im, 0.0006110808494121687 + 0.0im, 0.00012557561626010857 + 0.0im, 0.00014402240532780138 + 0.0im, 0.0002951783050394717 + 0.0im, 4.055832828662541e-5 + 0.0im, 0.0001062393578483567 + 0.0im, 7.91567111616861e-5 + 0.0im, 4.486765066708238e-5 + 0.0im  …  0.0005284839899797469 + 0.0im, 0.00016226599147984566 + 0.0im, 0.00010991273837586861 + 0.0im, 0.0001290705585573066 + 0.0im, 4.055832828662541e-5 + 0.0im, 0.00027057238743175997 + 0.0im, 6.526302748315783e-5 + 0.0im, 4.055832828662541e-5 + 0.0im, 0.0012171406601299613 + 0.0im, 0.00010561415510668609 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574837788480269e-14,).
), time = 7.46000825, bytes = 1596159245, gctime = 0.151902042, gcstats = Base.GC_Diff(1596159245, 213, 0, 445641, 241, 219, 151902042, 4, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [34]:
@assert abs(Real(vals[1])) < 1e-8

In [35]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [36]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 6-7 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

### Q6 Answer:
See below, the pages with top PageRank scores are not the same as the ranking according to in-degrees.

In [37]:
println("Index of pages with top 20 PageRank scores:", 
        reverse(sortperm(x_arnoldi))[1:20])

Index of pages with top 20 PageRank scores:[597622, 41910, 163076, 537040, 384667, 504141, 486981, 605857, 32164, 558792, 551830, 765335, 751385, 425771, 908352, 173977, 7315, 213433, 885606, 819224]


## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.